In [4]:
import numpy as np
import pandas as pd
from neo4j import GraphDatabase

In [5]:
# class DataBaseExample:

#     def __init__(self, uri, user, password):
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self.driver.close()

#     def print_greeting(self, message):
#         with self.driver.session() as session:
#             result = session.execute_write(self._run_cypher_code, message)
#             print(result)

#     @staticmethod
#     def _run_cypher_code(tx, message):
#         result = tx.run("CREATE (a:Greeting) "
#                         "SET a.message = $message "
#                         "RETURN a.message + ', from node ' + id(a)", message=message)
#         return result.single()[0]

In [3]:
# greeter = DataBaseExample("", "", "")
# greeter.print_greeting("hello, world")
# greeter.close()

hello, world, from node 0


In [33]:
# class DataBaseExample:

#     def __init__(self, uri, user, password):
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self.driver.close()

#     def create_node(self, label, **properties):
#         with self.driver.session() as session:
#             result = session.write_transaction(self._create_node, label, properties)
#             print(result)

#     def create_relationship(self, start_node_name, end_node_name, relationship_type, **properties):
#         with self.driver.session() as session:
#             result = session.write_transaction(self._create_relationship, start_node_name, end_node_name, relationship_type, properties)
#             print(result)

#     @staticmethod
#     def _create_node(tx, label, properties):
#         cypher = f"CREATE (n:{label} $properties) RETURN n"
#         result = tx.run(cypher, properties=properties)
#         return result.single()

#     @staticmethod
#     def _create_relationship(tx, start_node_name, end_node_name, relationship_type, properties):
#         cypher = (
#             "MATCH (start {name: $start_name}), (end {name: $end_name}) "
#             "CREATE (start)-[r:" + relationship_type + " $properties]->(end) "
#             "RETURN r"
#         )
#         result = tx.run(cypher, start_name=start_node_name, end_name=end_node_name, properties=properties)
#         return result.single()

In [34]:
# # Example usage
# uri = ""
# user = ""
# password = ""
# example = DataBaseExample(uri, user, password)

In [36]:
# example.create_node("Person", name="Alice", age=30)
# example.create_node("Person", name="Bob", age=35)

# # Create relationship
# example.create_relationship("Alice", "Bob", "KNOWS", since=2010)
# example.close()

C:\Users\ali20\AppData\Local\Temp/ipykernel_4320/550792496.py:10: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:
C:\Users\ali20\AppData\Local\Temp/ipykernel_4320/550792496.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  result = session.write_transaction(self._create_node, label, properties)


<Record n=<Node element_id='4:1f568fbe-2c5a-4c12-b680-bd2206ad2ca3:1' labels=frozenset({'Person'}) properties={'name': 'Alice', 'age': 30}>>
<Record n=<Node element_id='4:1f568fbe-2c5a-4c12-b680-bd2206ad2ca3:5' labels=frozenset({'Person'}) properties={'name': 'Bob', 'age': 35}>>
<Record r=<Relationship element_id='5:1f568fbe-2c5a-4c12-b680-bd2206ad2ca3:1152921504606846977' nodes=(<Node element_id='4:1f568fbe-2c5a-4c12-b680-bd2206ad2ca3:1' labels=frozenset() properties={}>, <Node element_id='4:1f568fbe-2c5a-4c12-b680-bd2206ad2ca3:5' labels=frozenset() properties={}>) type='KNOWS' properties={'since': 2010}>>


C:\Users\ali20\AppData\Local\Temp/ipykernel_4320/550792496.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:
C:\Users\ali20\AppData\Local\Temp/ipykernel_4320/550792496.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  result = session.write_transaction(self._create_relationship, start_node_name, end_node_name, relationship_type, properties)


In [37]:
class TemperatureDatabase:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def add_room(self, room_name):
        with self._driver.session() as session:
            session.write_transaction(self._create_room, room_name)

    def add_sensor(self, sensor_id, sensor_type):
        with self._driver.session() as session:
            session.write_transaction(self._create_sensor, sensor_id, sensor_type)

    def add_room_sensor_relationship(self, room_name, sensor_id):
        with self._driver.session() as session:
            session.write_transaction(self._create_room_sensor_relationship, room_name, sensor_id)

    def add_temperature_reading(self, sensor_id, value, timestamp):
        with self._driver.session() as session:
            session.write_transaction(self._create_temperature_reading, sensor_id, value, timestamp)

    @staticmethod
    def _create_room(tx, room_name):
        query = "CREATE (:Room {name: $room_name})"
        tx.run(query, room_name=room_name)

    @staticmethod
    def _create_sensor(tx, sensor_id, sensor_type):
        query = "CREATE (:Sensor {id: $sensor_id, type: $sensor_type})"
        tx.run(query, sensor_id=sensor_id, sensor_type=sensor_type)

    @staticmethod
    def _create_room_sensor_relationship(tx, room_name, sensor_id):
        query = (
            "MATCH (r:Room {name: $room_name}) "
            "MATCH (s:Sensor {id: $sensor_id}) "
            "MERGE (r)-[:CONTAINS]->(s)"
        )
        tx.run(query, room_name=room_name, sensor_id=sensor_id)

    @staticmethod
    def _create_temperature_reading(tx, sensor_id, value, timestamp):
        query = (
            "MATCH (s:Sensor {id: $sensor_id}) "
            "CREATE (s)-[:MEASURES]->(:TemperatureReading {value: $value, timestamp: $timestamp})"
        )
        tx.run(query, sensor_id=sensor_id, value=value, timestamp=timestamp)

In [39]:
uri = ""
user = ""
password = ""

db = TemperatureDatabase(uri, user, password)

In [40]:
# Add rooms
db.add_room('Kitchen')
db.add_room('Living Room')

# Add sensors
db.add_sensor('Sensor1', 'Temperature')
db.add_sensor('Sensor2', 'Temperature')

# Create relationships between rooms and sensors
db.add_room_sensor_relationship('Kitchen', 'Sensor1')
db.add_room_sensor_relationship('Living Room', 'Sensor2')

# Add temperature readings
db.add_temperature_reading('Sensor1', 21, '2024-04-09T10:00:00Z')
db.add_temperature_reading('Sensor2', 23, '2024-04-09T10:00:00Z')

db.close()

C:\Users\ali20\AppData\Local\Temp/ipykernel_4320/1356506378.py:10: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_room, room_name)
C:\Users\ali20\AppData\Local\Temp/ipykernel_4320/1356506378.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_sensor, sensor_id, sensor_type)
C:\Users\ali20\AppData\Local\Temp/ipykernel_4320/1356506378.py:18: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_room_sensor_relationship, room_name, sensor_id)
C:\Users\ali20\AppData\Local\Temp/ipykernel_4320/1356506378.py:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_temperature_reading, sensor_id, value, timestamp)
